In [1]:
mkdir simple_blockchain

In [2]:
cd simple_blockchain

/content/simple_blockchain


In [3]:
open("blockchain.py", "w").close()

In [4]:
import hashlib
import time
import json
import os

# -----------------------------
# Block Class
# -----------------------------
class Block:
    def __init__(self, index, transactions, previous_hash, timestamp=None):
        self.index = index
        self.timestamp = timestamp if timestamp else time.time()
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        value = str(self.index) + str(self.timestamp) + str(self.transactions) + str(self.previous_hash) + str(self.nonce)
        return hashlib.sha256(value.encode()).hexdigest()

    def mine_block(self, difficulty):
        print(f"⛏️ Mining block {self.index}...")
        target = '0' * difficulty
        while self.hash[:difficulty] != target:
            self.nonce += 1
            self.hash = self.calculate_hash()
        print(f"✅ Block {self.index} mined with hash: {self.hash}")

In [5]:
class Blockchain:
    def __init__(self, difficulty=2, max_tx_per_block=3):
        self.chain = []
        self.difficulty = difficulty
        self.pending_transactions = []
        self.max_tx_per_block = max_tx_per_block

        if os.path.exists("blockchain.json"):
            self.load_chain()
        else:
            self.create_genesis_block()

    def create_genesis_block(self):
        genesis_block = Block(0, ["Genesis Block"], "0")
        genesis_block.mine_block(self.difficulty)
        self.chain.append(genesis_block)
        self.save_chain()

    def get_latest_block(self):
        return self.chain[-1]

    def add_transaction(self, transaction_data):
        self.pending_transactions.append(transaction_data)
        if len(self.pending_transactions) >= self.max_tx_per_block:
            self.mine_pending_transactions()

    def mine_pending_transactions(self):
        latest_block = self.get_latest_block()
        new_block = Block(len(self.chain), self.pending_transactions[:self.max_tx_per_block], latest_block.hash)
        new_block.mine_block(self.difficulty)
        self.chain.append(new_block)
        self.pending_transactions = self.pending_transactions[self.max_tx_per_block:]
        self.save_chain()

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i - 1]

            if current.hash != current.calculate_hash():
                print(f"⚠️ Tampering detected at block {current.index}!")
                return False

            if current.previous_hash != previous.hash:
                print(f"⚠️ Block {current.index} not linked properly!")
                return False

        print("✅ Blockchain is valid.")
        return True

    def display_chain(self):
        for block in self.chain:
            print(f"Index: {block.index}")
            print(f"Timestamp: {time.ctime(block.timestamp)}")
            print(f"Transactions: {block.transactions}")
            print(f"Hash: {block.hash}")
            print(f"Previous Hash: {block.previous_hash}")
            print("-" * 40)

    def save_chain(self):
        with open("blockchain.json", "w") as f:
            json.dump([block.__dict__ for block in self.chain], f, indent=4)

    def load_chain(self):
        with open("blockchain.json", "r") as f:
            data = json.load(f)
            self.chain = []
            for block_data in data:
                block = Block(
                    block_data['index'],
                    block_data['transactions'],
                    block_data['previous_hash'],
                    block_data['timestamp']
                )
                block.nonce = block_data['nonce']
                block.hash = block_data['hash']
                self.chain.append(block)

In [ ]:
def menu():
    bc = Blockchain()

    while True:
        print("\n--- Blockchain Menu ---")
        print("1. Add new transaction")
        print("2. Mine pending transactions (if any)")
        print("3. Display blockchain")
        print("4. Validate blockchain")
        print("5. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            tx = input("Enter transaction data: ")
            bc.add_transaction(tx)
        elif choice == '2':
            if bc.pending_transactions:
                bc.mine_pending_transactions()
            else:
                print("🕒 No transactions to mine.")
        elif choice == '3':
            bc.display_chain()
        elif choice == '4':
            bc.is_chain_valid()
        elif choice == '5':
            break
        else:
            print("❌ Invalid choice. Try again.")

if __name__ == "__main__":
    menu()


⛏️ Mining block 0...
✅ Block 0 mined with hash: 0034cf17937ee45bf46c1ab87b717606262ea3d97f347d1d029dc713296002df

--- Blockchain Menu ---
1. Add new transaction
2. Mine pending transactions (if any)
3. Display blockchain
4. Validate blockchain
5. Exit
Enter your choice: 3
Index: 0
Timestamp: Sat Jul 19 15:45:33 2025
Transactions: ['Genesis Block']
Hash: 0034cf17937ee45bf46c1ab87b717606262ea3d97f347d1d029dc713296002df
Previous Hash: 0
----------------------------------------

--- Blockchain Menu ---
1. Add new transaction
2. Mine pending transactions (if any)
3. Display blockchain
4. Validate blockchain
5. Exit
Enter your choice: 1
Enter transaction data: Alice pays Bob 5 BTC

--- Blockchain Menu ---
1. Add new transaction
2. Mine pending transactions (if any)
3. Display blockchain
4. Validate blockchain
5. Exit
Enter your choice: 1
Enter transaction data: Charlie sends Dave 3 BTC

--- Blockchain Menu ---
1. Add new transaction
2. Mine pending transactions (if any)
3. Display blockchain